# Shor's Algorithm

Shor's algorithm accepts a (large) integer $N$ and finds $p$, $q$ such that $N=p\cdot q$ and both $p$, $q$ are prime

1. Choose a random integer $1 < a < N$
2. Compute $b = \gcd(a,N)$
3. If $b \neq 1$: $b$ factors $N$, return $p=b$ and $q=N/b$
4. Else:
5. Use quantum subroutine to find the order $r$ of $a$ (i.e., $\min(r>0)$ such that $a^r = 1\mod{N}$
6. If $r$ is odd: move back to line 1
7. Compute $g = \gcd(a^{r/2}\pm 1, N)$
8. If $g > 1$: return $p=g$, $q=N/g$
9. Else: go back to line 1

## Continued Fractions Algorithm

The Continued Fractions Algorithm finds the continued fraction expansion for any irrational number to as many partial denominators as desired

The following system of equations is a way of describing the continued fractions algorithm

$$
\begin{align*}
   x &= a_0 + b_0 \\
   \frac{1}{b_0} &= a_1' = a_1 + b_1 \\
   &\vdots\\
   \frac{1}{b_k} &= a_{k+1}' = a_{k+1} + b_{k+1}
\end{align*}
$$

Source: [Cornell](https://pi.math.cornell.edu/~gautam/ContinuedFractions.pdf)

**Input:**
- A number $x_0$

**Output:** $x_0 = \left[ a_0, a_1, a_2, ... \right]$

1. Let $a_m$ be the integer part of $x_m$ and $b_m \equiv x_m - a_m$
2. If $b_m \neq 0$ then:
3. &nbsp;&nbsp;&nbsp;&nbsp; Set $x_{m+1} = 1/b_m$ and go back to step 1 to compute $a_{m+1}$
4. Else then:
5. &nbsp;&nbsp;&nbsp;&nbsp; Return

In [35]:
import numpy as np

def continued_frac(x, eps=1e-8):
    
    def build_cf_representation(xm, a: np.ndarray):
        # Compute the integer part of xm
        am = int(xm)                      
        
        # Add integer part of xm to the results list (denominator of continued fractions)                                
        a = np.append(a, am)
        
        # Compute denominator of x_{m+1}                                  
        bm = xm - am                                                      
        
        # Terminate if bm ~ 0 
        if np.abs(bm) < eps:
            return a
        else:
            # Compute a_{m+1} 
            return build_cf_representation(1/bm, a)
        
    # Return the list of coefficients 
    return build_cf_representation(x, np.array([], dtype=int))    

assert (continued_frac(2.875) == [2,1,7]).all()

## Quantum Order-Finding Algorithm

https://github.com/qiskit-community/qiskit-community-tutorials/blob/master/algorithms/shor_algorithm.ipynb

This algorithm will take two coprime integers, $a$ and $N$, and output $r$ which is the period (i.e., $a^r\bmod{N}$)

**Input:**
- Two coprime integers $a$, $N$
- The operator $U$ such that $U\ket{x}\ket{y} = \ket{x}\ket{a^x\bmod N}$
- A state to store the function results, initialized to $\ket{0}$
- $n = L + \lceil \log{\left(2 + \frac{1}{2\epsilon}\right)} \rceil = O(L + \log{1/\epsilon})$ qubits initialized to $\ket{0}$ (and $L = 1 + 2\lceil\log{N}\rceil$)

**Output:** The period $r$ of $a^x\bmod N$

1. Create the initial state $\ket{0}\ket{0}$
2. Apply a Hadamard gate to the first register to create the superposition $\frac{1}{\sqrt{2^n}} \sum_{x=0}^{2^n-1}\ket{x}\ket{0}$
3. Apply the operator $U$ to get
$$
\frac{1}{\sqrt{2^n}} \sum_{x=0}^{2^n-1}\ket{x}\ket{a^x\bmod N} \approx \frac{1}{\sqrt{r2^n}} \sum_{x=0}^{2^n-1} \sum_{l=0}^{r-1} e^{2\pi ilx/r}\ket{x}\ket{u_l}
$$
    
&nbsp; where

$$
\ket{u_l} = \frac{1}{\sqrt{r}} \sum_{x=0}^{r-1}e^{-2\pi ilx/r}\ket{a^x\bmod N} \iff \ket{a^x\bmod N} = \frac{1}{\sqrt{r}} \sum_{l=0}^{r-1} e^{2\pi ilx/r}\ket{u_l}
$$

4. Apply the inverse Fourier transform on the first register to get
$$
\frac{1}{2^n\sqrt{r}} \sum_{l=0}^{r-1} \sum_{x=0}^{2^n-1} \sum_{k=0}^{2^n-1} e^{2\pi ilx/r} e^{-2\pi ixk/2^n} \ket{k} \ket{u_l} = \frac{1}{2^n\sqrt{r}} \sum_{l=0}^{r-1} \sum_{x=0}^{2^n-1} \sum_{k=0}^{2^n-1} e^{2\pi ix(l/r - k/2^n)} \ket{k} \ket{u_l}
$$

5. Measure the first register to get $k_0=2^n\frac{l}{r}$. The state will collapse to
$$
\frac{1}{2^n\sqrt{r}} \sum_{l=0}^{r-1} \sum_{x=0}^{2^n-1} e^{2\pi ix(l/r - k_0/2^n)} \ket{u_l} \equiv \frac{1}{\sqrt{r}} \sum_{l=0}^{r-1} \alpha_l \ket{u_l}
$$

&nbsp; with probability

$$
\mathcal{P}_{k_0} = \frac{1}{r} \sum_{l=0}^{r-1} \left| \alpha_l \right|^2
$$

6. Apply the continued fractions algorithm to determine $r$.

### Imports

In [60]:
from qiskit import ClassicalRegister, QuantumCircuit, QuantumRegister
import math
from qiskit.circuit.library import QFT

In [61]:
# inputs
# ----------------------
a, N = 13, 17
eps = 1e-3
# ----------------------

L = 1 + 2 * math.ceil( math.log(N, 2) )
n = L + math.ceil( math.log(2 + 1/2/eps, 2) )

# init
#First quantum register for Phase Estimation
#Classical register for storing measurementsialize two registers of size n
qar = QuantumRegister(2*n, name='argument reg.')
qfr = QuantumRegister(n, name='function reg.')
crr = ClassicalRegister(2*n, name='results reg.')

# initialize circuit
shor = QuantumCircuit(qar, qfr, crr)

#####

# apply Hadamard gate to first (argument) register
shor.h(qar)

shor.draw()


#Apply inverse QFT to the first Register
shor.append(QFT(2*n, inverse=True), qar)


#Measure the first register 
shor.measure(qar, crr)

#Execute the ciruit 
shor.draw()

┌───┐┌────────┐┌─┐                                          »
 argument reg._0: ┤ H ├┤0       ├┤M├──────────────────────────────────────────»
                  ├───┤│        │└╥┘┌─┐                                       »
 argument reg._1: ┤ H ├┤1       ├─╫─┤M├───────────────────────────────────────»
                  ├───┤│        │ ║ └╥┘┌─┐                                    »
 argument reg._2: ┤ H ├┤2       ├─╫──╫─┤M├────────────────────────────────────»
                  ├───┤│        │ ║  ║ └╥┘┌─┐                                 »
 argument reg._3: ┤ H ├┤3       ├─╫──╫──╫─┤M├─────────────────────────────────»
                  ├───┤│        │ ║  ║  ║ └╥┘┌─┐                              »
 argument reg._4: ┤ H ├┤4       ├─╫──╫──╫──╫─┤M├──────────────────────────────»
                  ├───┤│        │ ║  ║  ║  ║ └╥┘┌─┐                           »
 argument reg._5: ┤ H ├┤5       ├─╫──╫──╫──╫──╫─┤M├───────────────────────────»
                  ├───┤│        │ ║  ║  ║  ║  ║ └╥┘┌─┐                        »
 argument reg._6: ┤ H ├┤6       ├─╫──╫──╫──╫──╫──╫─┤M├────────────────────────»
                  ├───┤│        │ ║  ║  ║  ║  ║  ║ └╥┘┌─┐                     »
 argument reg._7: ┤ H ├┤7       ├─╫──╫──╫──╫──╫──╫──╫─┤M├─────────────────────»
                  ├───┤│        │ ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐                  »
 argument reg._8: ┤ H ├┤8       ├─╫──╫──╫──╫──╫──╫──╫──╫─┤M├──────────────────»
                  ├───┤│        │ ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐               »
 argument reg._9: ┤ H ├┤9       ├─╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├───────────────»
                  ├───┤│        │ ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐            »
argument reg._10: ┤ H ├┤10      ├─╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├────────────»
                  ├───┤│        │ ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐         »
argument reg._11: ┤ H ├┤11      ├─╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├─────────»
                  ├───┤│        │ ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐      »
argument reg._12: ┤ H ├┤12      ├─╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├──────»
                  ├───┤│        │ ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐   »
argument reg._13: ┤ H ├┤13      ├─╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├───»
                  ├───┤│        │ ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐»
argument reg._14: ┤ H ├┤14      ├─╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├»
                  ├───┤│        │ ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘»
argument reg._15: ┤ H ├┤15      ├─╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─»
                  ├───┤│        │ ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ »
argument reg._16: ┤ H ├┤16      ├─╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─»
                  ├───┤│        │ ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ »
argument reg._17: ┤ H ├┤17      ├─╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─»
                  ├───┤│        │ ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ »
argument reg._18: ┤ H ├┤18      ├─╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─»
                  ├───┤│        │ ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ »
argument reg._19: ┤ H ├┤19      ├─╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─»
                  ├───┤│   IQFT │ ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ »
argument reg._20: ┤ H ├┤20      ├─╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─»
                  ├───┤│        │ ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ »
argument reg._21: ┤ H ├┤21      ├─╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─»
                  ├───┤│        │ ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ »
argument reg._22: ┤ H ├┤22      ├─╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─»
                  ├───┤│        │ ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ »
argument reg._23: ┤ H ├┤23      ├─╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─»
                  ├───┤│        │ ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ »
argument reg._24: ┤ H ├┤24      ├─╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─»
                  